In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pfss.needs import *
from pfss.pfss_module import pfss_solver
from pfss.scs_module import scs_solver

## functions

In [47]:
def rk45(rfun, x, dl, **kwargs):
    sig = kwargs.get('sig', 1)
    x0  = x
    k1  = rfun(x0            , **kwargs)
    k2  = rfun(x0+sig*k1*dl/2, **kwargs)
    k3  = rfun(x0+sig*k2*dl/2, **kwargs)
    k4  = rfun(x0+sig*k3*dl  , **kwargs)
    k   = (k1+2*k2+2*k3+k4)/6*sig
    ret = x0+k*dl
    ret[1] = ret[1] % np.pi
    ret[2] = ret[2] %(np.pi*2)
    return ret

from pfss.funcs import trilinear_interpolation
def get_Brtp(rtp, **kwargs):
    Rcp   = kwargs.get('Rcp', instance.Rcp)
    r,t,p = rtp
    if r<Rcp:
        ir   = (r   - 1)/(instance.Rs-1)*(instance.n_r-1)
        it   = (np.pi-t)/np.pi*(instance.n_t-1)
        ip   = (p   - 0)/2/np.pi*(instance.n_p-1)
        idx  = [ir,it,ip]
        ret  = trilinear_interpolation(pfss.transpose(1,2,3,0), idx)
    if r>=Rcp:
        Nr,Nt,Np = instance.Nrtp_scs
        ir   = (r-instance.Rcp)/(instance.Rtp-instance.Rcp)*(Nr-1)
        it   = (np.pi-t)/np.pi*(Nt-1)
        ip   = (p-0)/2/np.pi*(Np-1)
        idx  = [ir,it,ip]
        ret  = trilinear_interpolation(scs.transpose(1,2,3,0), idx)
    return ret

def magline_stepper(rtp, **kwargs):
    # print('rtp :' ,rtp)
    eps   = kwargs.get('eps', 1e-10)
    if np.any(np.isnan(rtp)):
        return np.full(3,np.nan)
    r,t,p = rtp
    Brtp  = get_Brtp(rtp)
    Bn    = np.linalg.norm(Brtp)
    if Bn<eps:
        return np.full(3,np.nan)
    Bhat  = Brtp/np.linalg.norm(Brtp, axis=0)
    ret   = Bhat/np.array([1,r,r*np.sin(t)])
    return ret

def magline_solver(rtps, ntasks, dl=0.05, Rl=1.0, Rs=2.5, Ns=10000, lock=None,progress=None, t0=time.time()):
    rtps = np.array(rtps)
    maglines = []
    if rtps.ndim==1:
        rtps = rtps[None,:]
    for irtp in rtps:
        rtp0 = irtp
        forward  = [rtp0]
        backward = []
        # forward integral
        for i in range(Ns):
            if rtp0[0]<Rl or rtp0[0]>Rs or np.any(np.isnan(rtp0)):
                break
            rtp0 = rk45(magline_stepper, rtp0, dl, sig= 1)
            forward.append(rtp0)
        # backward integral
        rtp0 = irtp
        for i in range(Ns):
            if rtp0[0]<Rl or rtp0[0]>Rs or np.any(np.isnan(rtp0)):
                break
            rtp0 = rk45(magline_stepper, rtp0, dl, sig=-1)
            backward.append(rtp0)
        imagline = np.array(backward[::-1]+forward)
        maglines.append(imagline)
        with lock:
            ti = time.time()
            progress[0]+=1
            print(f"Tasks:{progress[0]:4d}/{ntasks:4d}, Line_point_nums: {len(imagline):5d}, Wall_time: {(ti-t0)/60:7.3f} min",
                  flush=True)
    return maglines


def parallel_magline_solver(rtps, **kwargs):
    t0 = time.time()
    dl   = kwargs.get('step_length', 0.05 )
    Rl   = kwargs.get('Rl'         , 1.0  )
    Rs   = kwargs.get('Rs'         , 10.0 )
    Ns   = kwargs.get('max_steps'  , 10000)
    total_tasks = len(rtps)
    n_cores     = min(kwargs.get('n_cores', 50), multiprocessing.cpu_count())
    chunks      = np.ceil(total_tasks/n_cores).astype(int)
    chunks_ls   = [chunks]*(total_tasks%n_cores)+[chunks-1]*(n_cores-total_tasks%n_cores)
    magline_res = []
    print('### =========== Parallel computing ============ ###')
    print(f'#       Available CPU cores: {multiprocessing.cpu_count():3d}                  #')
    print(f'#            Used CPU cores: {n_cores:3d}                  #')
    print('### =========================================== ###')

    manager     = Manager()
    progress    = manager.list([0])
    lock        = manager.Lock()

    with ProcessPoolExecutor(max_workers=n_cores) as executor:
        futures = []
        for i in range(n_cores):
            idx_i = np.sum(chunks_ls[:i]  ).astype(int)
            idx_f = np.sum(chunks_ls[:i+1]).astype(int)
            irtp  = rtps[idx_i:idx_f]
            futures.append(executor.submit(magline_solver, irtp, total_tasks, dl, Rl, Rs, Ns, lock, progress, t0))
        for future in futures:
            magline_res.extend(future.result())
    print(f'Time Used: {(time.time()-t0)/60:8.3f} min')
    return magline_res

## codes

In [14]:
load_file = 'scs_solver.pkl'
load_type = 'ss'

if load_type=='ss':
    instance = scs_solver.load(load_name=load_file)
elif load_type=='ps':
    instance = pfss_solver.load(load_name=load_file)
else:
    raise ValueError("-t option can only support to be 'ss' or 'ps'...")

In [8]:
mag_info = instance.info['maglines']
seeds    = mag_info['seeds']
n_cores  = mag_info.get('n_cores', 50)
eps      = mag_info.get('eps', 1e-10)
dl       = mag_info.get('step_length', 0.05)
Ns       = mag_info.get('max_steps', 10000)
Rl       = mag_info.get('Rl',1.0 )
Rs       = mag_info.get('Rs',instance.Rtp)

In [48]:
maglines = parallel_magline_solver(seeds, 
                                   step_length=dl,
                                   max_steps=Ns,
                                   Rl=Rl,
                                   Rs=Rs,
                                   n_cores=n_cores
                                  )

### =========== Parallel computing ============ ###
#       Available CPU cores:  96                  #
#            Used CPU cores:  70                  #
### =========================================== ###
Tasks:   1/ 100, Line_point_nums:   182, Wall_time:   0.036 min
Tasks:   2/ 100, Line_point_nums:   189, Wall_time:   0.037 min
Tasks:   3/ 100, Line_point_nums:   169, Wall_time:   0.037 min
Tasks:   4/ 100, Line_point_nums:   183, Wall_time:   0.037 min
Tasks:   5/ 100, Line_point_nums:   185, Wall_time:   0.037 min
Tasks:   6/ 100, Line_point_nums:   185, Wall_time:   0.037 min
Tasks:   7/ 100, Line_point_nums:   188, Wall_time:   0.037 min
Tasks:   8/ 100, Line_point_nums:   182, Wall_time:   0.037 min
Tasks:   9/ 100, Line_point_nums:   182, Wall_time:   0.037 min
Tasks:  10/ 100, Line_point_nums:   189, Wall_time:   0.037 min
Tasks:  11/ 100, Line_point_nums:   199, Wall_time:   0.037 min
Tasks:  12/ 100, Line_point_nums:   192, Wall_time:   0.037 min
Tasks:  13/ 100, Line_po

In [50]:
test = np.array(maglines, dtype=object)

In [59]:
np.save('maglines.npy', test)
test = np.load('maglines.npy', allow_pickle=True)

In [15]:
pfss = np.load(instance.pfss_file)
scs  = np.load(instance.scs_file)

In [30]:
pfss_rtp = pfss_solver.get_rtp(instance)
scs_rtp  = instance.get_rtp()

In [33]:
Rcp = instance.Rcp

In [39]:
from pfss.funcs import trilinear_interpolation
def get_Brtp(rtp, **kwargs):
    Rcp   = kwargs.get('Rcp', instance.Rcp)
    r,t,p = rtp
    if r<Rcp:
        ir   = (r   - 1)/(instance.Rs-1)*(instance.n_r-1)
        it   = (np.pi-t)/np.pi*(instance.n_t-1)
        ip   = (p   - 0)/2/np.pi*(insatnce.n_p-1)
        idx  = [ir,it,ip]
        ret  = trilinear_interpolation(pfss.transpose(1,2,3,0), idx)
    if r>=Rcp:
        Nr,Nt,Np = instance.Nrtp_scs
        ir   = (r-instance.Rcp)/(instance.Rtp-instance.Rcp)*(Nr-1)
        it   = (np.pi-t)/np.pi*(Nt-1)
        ip   = (p-0)/2/np.pi*(Np-1)
        idx  = [ir,it,ip]
        ret  = trilinear_interpolation(scs.transpose(1,2,3,0), idx)
    return ret

In [40]:
get_Brtp(seeds[0])

array([-0.05377058,  0.00439691,  0.00105606])